In [ ]:
import pandas as pd
import numpy as np

In [36]:
#val_results = pd.read_csv('../doc/models/model_02/model_02.csv')
val_results = pd.read_csv('../../doc/models/model_02r/model_02r.csv')
val_results.head()

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score
0,sub-100079,ses-V02,1,T2w,sub-100079_ses-V02_run-1_T2w.nii.gz,3.0,NaN
1,sub-100247,ses-V02,1,T2w,sub-100247_ses-V02_run-1_T2w.nii.gz,3.0,NaN
2,sub-100524,ses-V02,1,T2w,sub-100524_ses-V02_run-1_T2w.nii.gz,1.0,0.774361
3,sub-100586,ses-V02,1,T2w,sub-100586_ses-V02_run-1_T2w.nii.gz,0.0,NaN
4,sub-100642,ses-V02,1,T2w,sub-100642_ses-V02_run-1_T2w.nii.gz,0.0,NaN


In [37]:
# find scans with highest prediction error
val_results['abs_error'] = abs(val_results['QU_motion'] - val_results['predicted_qu_motion_score'])
val_results = val_results.sort_values(by='abs_error', ascending=False)
val_results.head(15)

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,abs_error
1023,sub-189059,ses-V02,2,T2w,sub-189059_ses-V02_run-2_T2w.nii.gz,3.0,-3.401781,6.401781
1022,sub-189059,ses-V02,1,T2w,sub-189059_ses-V02_run-1_T2w.nii.gz,3.0,-1.356314,4.356314
1016,sub-187858,ses-V02,3,T1w,sub-187858_ses-V02_run-3_T1w.nii.gz,3.0,0.811780,2.188220
1014,sub-187858,ses-V02,1,T1w,sub-187858_ses-V02_run-1_T1w.nii.gz,3.0,0.828692,2.171308
587,sub-151084,ses-V03,2,T1w,sub-151084_ses-V03_run-2_T1w.nii.gz,3.0,0.913929,2.086071
561,sub-148500,ses-V02,2,T1w,sub-148500_ses-V02_run-2_T1w.nii.gz,3.0,0.982758,2.017242
534,sub-146372,ses-V02,1,T2w,sub-146372_ses-V02_run-1_T2w.nii.gz,3.0,1.122338,1.877662
158,sub-115876,ses-V02,1,T1w,sub-115876_ses-V02_run-1_T1w.nii.gz,0.5,2.251156,1.751156
331,sub-132848,ses-V02,1,T1w,sub-132848_ses-V02_run-1_T1w.nii.gz,3.0,1.272246,1.727754
504,sub-144495,ses-V02,1,T2w,sub-144495_ses-V02_run-1_T2w.nii.gz,3.0,1.490790,1.509210


In [38]:
# save first 15 rows to csv
val_results.head(15).to_csv('../../data/validation_errors/model_02r/validation_set_top15_errors.csv', index=False)

In [39]:
# remove the 5 highest error scans
val_results = val_results.iloc[5:]
val_results.head()

,subject_id,session_id,run_id,suffix,scan,QU_motion,predicted_qu_motion_score,abs_error
561,sub-148500,ses-V02,2,T1w,sub-148500_ses-V02_run-2_T1w.nii.gz,3.0,0.982758,2.017242
534,sub-146372,ses-V02,1,T2w,sub-146372_ses-V02_run-1_T2w.nii.gz,3.0,1.122338,1.877662
158,sub-115876,ses-V02,1,T1w,sub-115876_ses-V02_run-1_T1w.nii.gz,0.5,2.251156,1.751156
331,sub-132848,ses-V02,1,T1w,sub-132848_ses-V02_run-1_T1w.nii.gz,3.0,1.272246,1.727754
504,sub-144495,ses-V02,1,T2w,sub-144495_ses-V02_run-1_T2w.nii.gz,3.0,1.490790,1.509210


In [40]:
# count NaN values in abs_error column
val_results['abs_error'].isna().sum()

np.int64(814)

In [41]:
# drop rows with NaN values in abs_error column
val_results = val_results.dropna(subset=['abs_error'])
val_results['abs_error'].isna().sum()

np.int64(0)

In [44]:
def create_correlation_coefficient(actual_vals, predicted_vals):
    """Calculate Pearson correlation coefficient between actual and predicted values.

    Args:
        actual_vals (list): Actual QU_motion scores
        predicted_vals (list): Predicted QU_motion scores

    Returns:
        float: Pearson correlation coefficient (r value)
    """
    x = np.array(actual_vals)
    y = np.array(predicted_vals)

    correlation_matrix = np.corrcoef(x, y)
    correlation_coefficient = correlation_matrix[0, 1]

    return correlation_coefficient

In [ ]:
# recalculate metrics without the 5 highest error scans
corr = create_correlation_coefficient(val_results['QU_motion'], val_results['predicted_qu_motion_score'])
corr

np.float64(0.7748933033837027)